In [48]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array


result = pd.read_csv('preprocessed_data.csv')
print(result)


        선종  상태  state  vessel_type  nation_type         filename
0     낚시어선  조업      0            8            1   1. 낚시어선 조업.jpg
1      범장망  조업      0            3            0    1. 범장망 조업.jpg
2     낚시어선  표류      2            8            1   1. 낚시어선 표류.jpg
3     등광조망  이동      1            2            0   1. 등광조망 이동.jpg
4     등광조망  표류      2            2            0   1. 등광조망 표류.jpg
...    ...  ..    ...          ...          ...              ...
1576   안강망  조업      0            4            1    1. 안강망 조업.jpg
1577    연승  표류      2            5            1     1. 연승 표류.jpg
1578  등광조망  조업      0            2            0  10. 등광조망 조업.jpg
1579  등광조망  조업      0            2            0  10. 등광조망 조업.jpg
1580  등광조망  이동      1            2            0   1. 등광조망 이동.jpg

[1581 rows x 6 columns]


In [49]:
# 모델 불러오기
from tensorflow.keras.models import load_model
saved_model = load_model('mobilenet_model.h5')

In [50]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

# CSV 파일 경로
csv_file_path = 'preprocessed_data.csv'  # CSV 파일 경로
# 이미지 디렉토리 경로
image_dir = 'cropped_images'  # 이미지가 저장된 폴더 경로

# CSV 파일에서 이미지 파일명 읽기
df = pd.read_csv(csv_file_path)

# 'filename' 열에 저장된 이미지 파일명을 리스트로 가져옴
image_filenames = df['filename'].tolist()

# 이미지 읽기 및 전처리
images = []
for filename in image_filenames:
    if isinstance(filename, str) and filename.endswith(('.jpg', '.png')):  # 이미지 확장자 필터
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            images.append(image_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)

# 결과 출력
print(f"Processed {len(images)} images.")


Processed 1581 images.


In [51]:
target = result['vessel_type']
target = to_categorical(target, num_classes=10)  
target

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [59]:
from tensorflow.keras import layers, Model

def MobileNetV2(input_shape=(256, 256, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    
    # MobileNetV2 backbone
    backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    # backbone = tf.keras.applications.ResNet101V2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    # backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    
    # Add custom head for classification
    x = layers.GlobalAveragePooling2D()(backbone)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name='MobileNetV2')
    return model

model = MobileNetV2()
model.summary()



Model: "MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet101v2 (Functional)        │ (None, 8, 8, 2048)     │    42,626,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,153,674 (164.62 MB)

 Trainable params: 43,056,010 (164.25 MB)

 Non-trainable params: 97,664 (381.50 KB)

In [60]:
from tensorflow.keras.optimizers import Adam, SGD
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [61]:
history = model.fit(images, target, epochs=10, verbose=1)
predictions = model.predict(images)
# binary_predictions = (predictions > 0.5).astype(int)
class_predictions = np.argmax(predictions, axis=1)

print(class_predictions)

Epoch 1/10


2024-10-24 08:01:28.234683: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_30112', 4 bytes spill stores, 4 bytes spill loads

2024-10-24 08:01:29.732642: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_30330', 4 bytes spill stores, 4 bytes spill loads

2024-10-24 08:01:29.752704: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_30112', 308 bytes spill stores, 604 bytes spill loads

2024-10-24 08:01:30.909515: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_30330', 4 bytes spill stores, 4 bytes spill loads

2024-10-24 08:01:31.381986: I external/local_xla/xla/stream_

49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.8177 - loss: 0.6196

2024-10-24 08:02:39.181341: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_30112', 4 bytes spill stores, 4 bytes spill loads



50/50 ━━━━━━━━━━━━━━━━━━━━ 174s 1s/step - accuracy: 0.8216 - loss: 0.6086
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.7826 - loss: 1.0771
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 0.7145 - loss: 0.8999
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.8264 - loss: 0.6134
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.9189 - loss: 0.2759
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.9409 - loss: 0.2167
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.9240 - loss: 0.2054
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.9797 - loss: 0.1148
Epoch 9/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 0.9872 - loss: 0.0686
Epoch 10/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.9915 - loss: 0.0792
50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 168ms/step
[8 3 8 ... 2 2 2]


In [ ]:
# 모델 저장
model.save('mobilenet_model.h5')  # HDF5 형식으로 저장




In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

# 이미지 디렉토리 경로
image_dir = 'cropped_images'  # 경로 수정 필요

# 이미지 읽기 및 전처리
test_images = []
filenames = []
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            test_images.append(image_array)
            filenames.append(filename)  # 파일 이름을 저장하여 나중에 매칭
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
test_images = np.array(test_images)

# 예측 수행
prediction = model.predict(test_images)

# 클래스 예측값을 가져옴 (argmax로 각 이미지에 대해 가장 높은 확률의 클래스 선택)
class_predictions = np.argmax(prediction, axis=1)

# 이미지 파일 이름과 예측 결과를 매칭하여 출력
for i, filename in enumerate(filenames):
    print(f"Image: {filename} - Predicted Class: {class_predictions[i]}")


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Image: 1. 낚시어선 조업.jpg - Predicted Class: 3
Image: 1. 범장망 조업.jpg - Predicted Class: 3
Image: 1. 낚시어선 표류.jpg - Predicted Class: 3
Image: 1. 등광조망 이동.jpg - Predicted Class: 3
Image: 1. 등광조망 표류.jpg - Predicted Class: 3
Image: 1. 범장망 이동.jpg - Predicted Class: 3
Image: 1. 범장망 표류.jpg - Predicted Class: 3
Image: 1. 안강망 이동.jpg - Predicted Class: 3
Image: 1. 안강망 조업.jpg - Predicted Class: 3
Image: 1. 안강망 표류.jpg - Predicted Class: 3
Image: 1. 연승 이동.jpg - Predicted Class: 3
Image: 1. 유망 이동.jpg - Predicted Class: 3
Image: 1. 유망 조업.jpg - Predicted Class: 3
Image: 1. 유망 표류.jpg - Predicted Class: 3
Image: 1. 저인망 이동.jpg - Predicted Class: 3
Image: 1. 저인망 조업.jpg - Predicted Class: 3
Image: 1. 저인망 표류.jpg - Predicted Class: 3
Image: 1. 낚시어선 이동.jpg - Predicted Class: 3
Image: 1. 연승 표류.jpg - Predicted Class: 9
Image: 1. 채낚기  표류.jpg - Predicted Class: 3
Image: 1. 채낚기 조업.jpg - Predicted Class: 3
Image: 1. 채낚기 이동.jpg - Predicted Class: 3
Image: 1. 타망 이동.jpg - Predicted Cla

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

# 이미지 디렉토리 경로
image_dir = 'mini_test'  # 경로 수정 필요

# 클래스 레이블 매핑
class_labels = {
    0: '타망',
    1: '유망',
    2: '등광조망',
    3: '범장망',
    4: '안강망',
    5: '연승',
    6: '통발',
    7: '저인망',
    8: '낚시어선',
    9: '채낚기'
}

# 이미지 읽기 및 전처리
test_images = []
filenames = []
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            test_images.append(image_array)
            filenames.append(filename)  # 파일 이름을 저장하여 나중에 매칭
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
test_images = np.array(test_images)

# 예측 수행
prediction = model.predict(test_images)

# 클래스 예측값을 가져옴 (argmax로 각 이미지에 대해 가장 높은 확률의 클래스 선택)
class_predictions = np.argmax(prediction, axis=1)

# 이미지 파일 이름과 예측 결과를 매칭하여 출력
for i, filename in enumerate(filenames):
    predicted_class_name = class_labels[class_predictions[i]]  # 클래스 인덱스를 이름으로 매핑
    print(f"Image: {filename} - Predicted Class: {predicted_class_name}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Image: 1. 저인망 조업.jpg - Predicted Class: 범장망
Image: 1. 타망 조업.jpg - Predicted Class: 범장망
Image: 1.등광조망 조업.jpg - Predicted Class: 범장망
Image: 11. 유망 조업.jpg - Predicted Class: 범장망
Image: 23. 등광조망 조업.jpg - Predicted Class: 범장망
Image: 5. 등광조망 조업.jpg - Predicted Class: 범장망
Image: 5. 저인망 조업.jpg - Predicted Class: 범장망
Image: 5. 타망 조업.jpg - Predicted Class: 범장망
Image: 6. 낚시어선 이동.jpg - Predicted Class: 범장망
Image: 7. 유망 조업.jpg - Predicted Class: 범장망
Image: 9. 등광조망 조업.jpg - Predicted Class: 범장망
Image: 9. 유망 조업.jpg - Predicted Class: 범장망
Image: 9. 저인망 조업.jpg - Predicted Class: 범장망
Image: 9. 타망 조업.jpg - Predicted Class: 범장망


타망: 0
유망: 1
등광조망: 2
범장망: 3
안강망: 4
연승: 5
통발: 6
저인망: 7
낚시어선: 8
채낚기: 9